In [1]:
import numpy as np
import tensorflow as tf

In [2]:
# build data
input_x = np.random.rand(1000, 4)
output_y = np.dot(input_x,np.array([[3],[14],[6],[10]]))

In [3]:
print(input_x.shape)
print(output_y.shape)

(1000, 4)
(1000, 1)


## Train Model

In [4]:
# build model
inputs = tf.keras.layers.Input(shape=(4,));
outputs = tf.keras.layers.Dense(units=1)(inputs)
model = tf.keras.Model(inputs=inputs,outputs=outputs)

In [5]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01),loss='mse')

In [6]:
model.fit(x=input_x,y=output_y,epochs=100,validation_split=0.1)

Epoch 1/100
29/29 [==============================] - 0s 5ms/step - loss: 125.7776 - val_loss: 33.4725
Epoch 2/100
29/29 [==============================] - 0s 2ms/step - loss: 20.2184 - val_loss: 10.6941
Epoch 3/100
29/29 [==============================] - 0s 2ms/step - loss: 10.2662 - val_loss: 8.1066
Epoch 4/100
29/29 [==============================] - 0s 2ms/step - loss: 8.7544 - val_loss: 7.4363
Epoch 5/100
29/29 [==============================] - 0s 2ms/step - loss: 8.0886 - val_loss: 6.9095
Epoch 6/100
29/29 [==============================] - 0s 2ms/step - loss: 7.5212 - val_loss: 6.4374
Epoch 7/100
29/29 [==============================] - 0s 2ms/step - loss: 7.0059 - val_loss: 6.0186
Epoch 8/100
29/29 [==============================] - 0s 2ms/step - loss: 6.5436 - val_loss: 5.6127
Epoch 9/100
29/29 [==============================] - 0s 1ms/step - loss: 6.1083 - val_loss: 5.2515
Epoch 10/100
29/29 [==============================] - 0s 2ms/step - loss: 5.7159 - val_loss: 4.9113
Epo

Epoch 83/100
29/29 [==============================] - 0s 2ms/step - loss: 0.1147 - val_loss: 0.1057
Epoch 84/100
29/29 [==============================] - 0s 2ms/step - loss: 0.1096 - val_loss: 0.1013
Epoch 85/100
29/29 [==============================] - 0s 1ms/step - loss: 0.1045 - val_loss: 0.0969
Epoch 86/100
29/29 [==============================] - 0s 2ms/step - loss: 0.0998 - val_loss: 0.0922
Epoch 87/100
29/29 [==============================] - 0s 2ms/step - loss: 0.0951 - val_loss: 0.0879
Epoch 88/100
29/29 [==============================] - 0s 2ms/step - loss: 0.0907 - val_loss: 0.0840
Epoch 89/100
29/29 [==============================] - 0s 2ms/step - loss: 0.0866 - val_loss: 0.0802
Epoch 90/100
29/29 [==============================] - 0s 1ms/step - loss: 0.0827 - val_loss: 0.0765
Epoch 91/100
29/29 [==============================] - 0s 1ms/step - loss: 0.0790 - val_loss: 0.0729
Epoch 92/100
29/29 [==============================] - 0s 2ms/step - loss: 0.0753 - val_loss: 0.0698


In [7]:
model.get_weights()

[array([[ 2.669988],
        [13.562156],
        [ 5.622848],
        [ 9.587276]], dtype=float32),
 array([0.80391794], dtype=float32)]

## save as pb
https://github.com/leimao/Frozen_Graph_TensorFlow

In [9]:
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

In [10]:
# Convert Keras model to ConcreteFunction
full_model = tf.function(lambda x: model(x))
full_model = full_model.get_concrete_function(
    tf.TensorSpec(model.inputs[0].shape, model.inputs[0].dtype))

# Get frozen ConcreteFunction
frozen_func = convert_variables_to_constants_v2(full_model)
frozen_func.graph.as_graph_def()

layers = [op.name for op in frozen_func.graph.get_operations()]
print("-" * 50)
print("Frozen model layers: ")
for layer in layers:
    print(layer)

print("-" * 50)
print("Frozen model inputs: ")
print(frozen_func.inputs)
print("Frozen model outputs: ")
print(frozen_func.outputs)

# Save frozen graph from frozen ConcreteFunction to hard drive
tf.io.write_graph(graph_or_graph_def=frozen_func.graph,
                  logdir="./frozen_models",
                  name="frozen_graph.pb",
                  as_text=False)

--------------------------------------------------
Frozen model layers: 
x
model/dense/MatMul/ReadVariableOp/resource
model/dense/MatMul/ReadVariableOp
model/dense/MatMul
model/dense/BiasAdd/ReadVariableOp/resource
model/dense/BiasAdd/ReadVariableOp
model/dense/BiasAdd
Identity
--------------------------------------------------
Frozen model inputs: 
[<tf.Tensor 'x:0' shape=(None, 4) dtype=float32>]
Frozen model outputs: 
[<tf.Tensor 'Identity:0' shape=(None, 1) dtype=float32>]


'./frozen_models/frozen_graph.pb'